In [ ]:
# default_exp core

# Auto ML Classidfication

> Automatically trains Models for Classification

In [1]:
#hide
from nbdev.showdoc import *

In [2]:
#export 
import streamlit as st
import streamlit.components.v1 as components
from pdpbox import pdp

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import shap
# load JS visualization code to notebook
shap.initjs()

import base64

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, load_digits

#Simple Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#Tree based Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier

#Gradient Based Classifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

#Preprocessing packages
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

#Metrics
from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
import random

import os
import base64
from io import BytesIO


def model_predict(data_asarray):
    data_asframe =  pd.DataFrame(data_asarray, columns=my_cols)
    return gscv.predict_proba(data_asframe)


def combined_metrics(X_test, y_test, clf):
    #to be used in combined metrics function.
#     enc = LabelEncoder()
#     trans_y_train = enc.fit_transform(y_train)
    ohe = OneHotEncoder(handle_unknown='ignore')
    ohe.fit(y_train.values.reshape(-1, 1))
    y_test_ohe = ohe.transform(y_test.values.reshape(-1, 1)).toarray()
    
    metrics_list = [[accuracy_score(y_test, 
                                    clf.predict(X_test))], 
                    [precision_score(y_test, clf.predict(X_test), average = 'micro')],

                    [recall_score(y_test, clf.predict(X_test), average = 'micro')], 
                    
                    [f1_score(y_test, clf.predict(X_test), average = 'micro')],
                    
                    [roc_auc_score(y_test_ohe, ohe.transform(clf.predict(X_test).reshape(-1, 1)).toarray(), multi_class='ovr')],
                    
                    [hamming_loss(y_test, clf.predict(X_test))], 
                   
                    [log_loss(y_test_ohe, ohe.transform(clf.predict(X_test).reshape(-1, 1)).toarray())]
                   ]
    

    index = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC', 'Hamming Loss', 'Log Loss']
#     index = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Hamming Loss', 'Log Loss']
#     index = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC', 'Hamming Loss']

    df_metric = pd.DataFrame(metrics_list, index = index, columns = ['Value'])

    return df_metric


def confusion_matrix_plot(cm, class_names, title = 'Confusion Matrix Plot'):
    plt.clf()
    plt.imshow(cm, interpolation='nearest', cmap = 'Blues_r')
    plt.title(title)
    plt.ylabel('True')
    plt.xlabel('Predicted')
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    s = [['TN','FP'], ['FN', 'TP']]
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j,i, str(cm[i][j]))
    plt.show()
            

def to_excel(df):
    output = BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    df.to_excel(writer, index = False, sheet_name='Sheet1')
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    format1 = workbook.add_format({'num_format': '0.00'}) # Tried with '0%' and '#,##0.00' also.
    worksheet.set_column('A:A', None, format1) # Say Data are in column A
    writer.save()
    processed_data = output.getvalue()
    return processed_data

def get_table_download_link(df):
    """Generates a link allowing the data in a given panda dataframe to be downloaded
    in:  dataframe
    out: href string
    """
    val = to_excel(df)
    b64 = base64.b64encode(val)  # val looks like b'...'
    return f'<a href="data:application/octet-stream;base64,{b64.decode()}" download="Your_File.xlsx">Download output file</a>' # decode b'abc' => abc


def GNB():
    gnb_params = {'clf__estimator':[GaussianNB()]
                 }
    return gnb_params
    
def LogisticReg():
    lr_params = {'clf__estimator': [LogisticRegression()]
                    }
    
    st.subheader('Logistic Regression')
    penalty = st.multiselect('Penalty', ['l1', 'l2'], ['l2'])
    reg = st.multiselect('C', [0.1, 1.0, 2.0], [1.0])
    solver = st.multiselect('Solver', ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], ['liblinear'])
    lr_params['clf__estimator__penalty'] = penalty
    lr_params['clf__estimator__C'] = reg
    lr_params['clf__estimator__solver'] = solver
    return lr_params
    
def KNN():
    knn_params = {'clf__estimator': [KNeighborsClassifier()]
                 }
    
    st.subheader('KNN')
    n_neighbors = st.multiselect('Neighbors', list(range(1,30)), [5])
    leaf_size = st.multiselect('Leaf Size', list(range(1,50)), [30])
    p_distance = st.multiselect('Distance Metric', [1,2], [2])
    
    knn_params['clf__estimator__n_neighbors'] = n_neighbors
    knn_params['clf__estimator__leaf_size'] = leaf_size
    knn_params['clf__estimator__p'] = p_distance
    return knn_params

def SVM():
    svm_params = {'clf__estimator': [SVC(probability=True)]
                 }
    
    st.subheader('Support Vector Machines')
    c = st.multiselect('C', [0.1, 1, 10, 100, 1000], [1])
    gamma = st.multiselect('Gamma', ['scale', 'auto'], ['scale'])
    kernel = st.multiselect('Kernel', ['linear', 'rbf', 'poly', 'sigmoid'], ['rbf'])
    
    svm_params['clf__estimator__C'] = c
    svm_params['clf__estimator__gamma'] = gamma
    svm_params['clf__estimator__kernel'] = kernel
    return svm_params
    

def DT():
    dt_params = {'clf__estimator': [DecisionTreeClassifier()]}
    
    st.subheader('Decision Tree')
    criterion = st.multiselect('Criterion', ["gini", "entropy"], ['gini'])
    min_samp_split = st.multiselect('Min Samples Split', [2, 10], [2])
    max_depth = st.multiselect('Max Depth', [2, 5, 10], [10])
    
    dt_params['clf__estimator__criterion'] = criterion
    dt_params['clf__estimator__min_samples_leaf'] = min_samp_split
    dt_params['clf__estimator__max_depth'] = max_depth
    return dt_params
    
def RF():
    rf_params = {'clf__estimator': [RandomForestClassifier()]
                }
    
    st.subheader('Random Forest')
    n_estimators = st.multiselect('Number of Trees', [100, 200, 500], [100]) 
    max_features = st.multiselect('Max Features', [2, 10, 'auto', 'sqrt', 'log2'], ['auto'])
    max_depth = st.multiselect('Max Depth', [4,5,6,7,8, None], [None])
    criterion = st.multiselect('Criteria', ['gini', 'entropy'], ['gini'])
    
    rf_params['clf__estimator__n_estimators'] = n_estimators
    rf_params['clf__estimator__max_features'] = max_features 
    rf_params['clf__estimator__max_depth'] = max_depth
    rf_params['clf__estimator__criterion'] = criterion
    return rf_params

def GB():
    gb_params = {'clf__estimator': [GradientBoostingClassifier()]
            }
    
    st.subheader('Gradient Booster')
    loss = st.multiselect('Loss Function', ['deviance', 'exponential'], ['deviance'])
    learning_rate = st.multiselect('Learning Rate', [0.001, 0.01, 0.1], [0.1])
    min_samples_split = st.multiselect('Min Samples Split', list(range(1, 10)), [2])
    min_samples_leaf = st.multiselect('Min Samples Leaf', list(range(1, 10)), [1])
    max_depth = st.multiselect('Max Depth', [1, 2, 3, 4, 5, 6], [3])
    max_features = st.multiselect('Max Features', ['auto', 'log2', 'sqrt', None], [None])
    criterion = st.multiselect('Criterion', ['friedman_mse', 'mse', 'mae'], ['friedman_mse'])
    subsample = st.multiselect('Subsample', [0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0], [1.0])
    n_estimators = st.multiselect('Number of Trees', [50, 100, 150, 200, 250], [100])
    
    gb_params['clf__estimator__loss'] = loss
    gb_params['clf__estimator__learning_rate'] = learning_rate
    gb_params['clf__estimator__min_samples_split'] = min_samples_split
    gb_params['clf__estimator__min_samples_leaf'] = min_samples_leaf
    gb_params['clf__estimator__max_depth'] = max_depth
    gb_params['clf__estimator__max_features']  = max_features
    gb_params['clf__estimator__criterion'] = criterion
    gb_params['clf__estimator__subsample'] = subsample
    gb_params['clf__estimator__n_estimators'] = n_estimators
    return gb_params
    
    
def ERT():
    ert_params = {'clf__estimator': [ExtraTreesClassifier()]
                 }
    
    st.subheader('Extra Random Trees')
    n_estimators = st.multiselect('Number of Trees', [100, 200, 500, 1000], [100]) #fix
    max_depth = st.multiselect('Max Depth', [None, 4, 5, 6, 7, 8, 9], [None])  #fix
    min_samples_leaf = st.multiselect('Min Sample per Leaf', [1, 2, 3, 4, 5], [1])
    n_jobs = st.selectbox('Parallelism', [1, 2, 3, 4, -1], 4) 
    
    ert_params['clf__estimator__n_estimators'] = n_estimators
    ert_params['clf__estimator__max_depth'] = max_depth
    ert_params['clf__estimator__min_samples_leaf'] = min_samples_leaf
    ert_params['clf__estimator__n_jobs'] = [n_jobs]
    return ert_params
    
def XGB():
    xgb_params ={'clf__estimator':[XGBClassifier()]
                }
    
    st.subheader('XGBoost')
    n_estimators = st.multiselect('Number of Trees', list(range(50, 1000, 50)), [50]) #fix
    max_depth = st.multiselect('Max Depth', list(range(1, 20)), [6])  #fix
    min_child_weight = st.multiselect('Min Child Weight', list(range(1, 10, 1)), [1])
    gamma = st.multiselect('Gamma', list(range(0, 10)), [1]) 
    learning_rate = st.multiselect('Learning Rate', [0.01, 0.05, 0.1, 0.2, 0.3], [0.3])
    subsample = st.multiselect('Subsample', list(np.divide(range(5, 11), 10)), [1.0])
    booster = st.multiselect('Booster', ['gbtree', 'gblinear'], ['gbtree'])
    
    xgb_params['clf__estimator__n_estimators'] = n_estimators
    xgb_params['clf__estimator__max_depth'] = max_depth
    xgb_params['clf__estimator__min_child_weight'] =  min_child_weight
    xgb_params['clf__estimator__gamma'] = gamma 
    xgb_params['clf__estimator__learning_rate'] = learning_rate
    xgb_params['clf__estimator__subsample'] = subsample
    xgb_params['clf__estimator__booster'] = booster
    return xgb_params
        
def SGD():
    sgd_params = {'clf__estimator': [SGDClassifier()]
                 }
    
    st.subheader('SGD')
    loss = st.multiselect('Loss Function', ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'], ['hinge']) #fix
    max_iter = st.multiselect('Max Iterations', list(np.multiply(range(5, 16),  100)), [1000])  #fix
    tol = st.multiselect('Tolerance', [0.0001, 0.001, 0.05, 0.1], [0.0001])
    penalty = st.multiselect('Penalty', ['l2', 'l1', 'elasticnet'], ['l2']) 
    alpha = st.multiselect('Alpha', [0.0001, 0.001, 0.05, 0.1, 0.2, 0.3], [0.0001])
    n_jobs = st.selectbox('Parallelization', [1, 2, 3, 4, -1], 4)
    
    sgd_params['clf__estimator__loss'] = loss
    sgd_params['clf__estimator__max_iter'] = max_iter
    sgd_params['clf__estimator__tol'] = tol
    sgd_params['clf__estimator__penalty'] = penalty
    sgd_params['clf__estimator__alpha'] = alpha
    sgd_params['clf__estimator__n_jobs'] = [n_jobs]
    return sgd_params

def NN():
    nn_params = {'clf__estimator': [MLPClassifier()]
                }
    
    st.subheader('Neural Network')
    solver = st.multiselect('Solver', ['lbfgs', 'sgd', 'adam'], ['adam'])
    max_iter = st.multiselect('Max Iterations', [1000,1100,1200,1300,1400], [1000])
    alpha = st.multiselect('Alpha', list(10.0 ** -np.arange(1, 10)), [0.0001])
    hidden_layer_sizes = st.multiselect('Hidden Layer Sizes', list(range(50, 500, 50)), [100])
#     hidden_layer_sizes = st.multiselect('Hidden Layer Sizes', [50, 100, 150, 200, 250, 300, 350, 400, 450, 500] , [100])
    
    nn_params['clf__estimator__solver'] = solver
    nn_params['clf__estimator__max_iter'] = max_iter
    nn_params['clf__estimator__alpha'] = alpha
    nn_params['clf__estimator__hidden_layer_sizes'] = hidden_layer_sizes
    return nn_params


data = st.file_uploader('Upload a csv')
test_data = st.file_uploader('Upload a csv for prediction:') 

if (data != None) & (test_data != None):
    df = pd.read_csv(data)
    df_test = pd.read_csv(test_data)
#     df = random.shuffle(data)
    target_col =st.selectbox('Choose target variable', df.columns)
    X = df.drop(target_col, axis = 1)
    y = df[target_col]
    test_ratio = st.number_input('Enter test split ratio, 0 < ratio < 1', min_value = 0.0, 
                                 max_value = 1.0, value = 0.2)
    
    if test_ratio:
        X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y,
                                                                        test_size=test_ratio,
                                                                        random_state = 0)
        
        enc = LabelEncoder()
        trans_y_train = enc.fit_transform(y_train)

        selected_models = st.sidebar.multiselect(
                              'Choose Algorithms:',(
                                  'Gaussian NB',
                                  'Logistic Regression', 
                                  'KNN',
                                  'Support Vector Machines',
                                  'Decision Tree',
                                  'Random Forest',
                                  'Gradient Boosting',
                                  'Extra Random Trees',
                                  'XGBoost',
                                  'Stochastic Gradient Descent',
                                  'Neural Network'), ['KNN', 'Support Vector Machines', 'Decision Tree'])
        
#         selected_models = st.sidebar.multiselect(
#                           'Choose Algorithms:',(
#                               'Gaussian NB',
#                               'Logistic Regression',))


    
        if selected_models:
#         gnb_params = {'clf__estimator': [GaussianNB()]
#                      }
#         lr_params = {'clf__estimator': [LogisticRegression()]
#                     }

#         st.subheader('Logistic Regression')
#         penalty = st.multiselect('Penalty', ['l1', 'l2'])
#         reg = st.multiselect('C', [0.1, 1.0, 2.0], [1.0])
#         solver = st.multiselect('Solver', ['liblinear', 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], ['liblinear'])
        
#         lr_params = LogisticReg()
#         param_dict = {'Gaussian NB': gnb_params,
#                           'Logistic Regression':lr_params, 
#          
    
            func_dict = {'Gaussian NB': GNB(),
                         'Logistic Regression':LogisticReg(), 
                          'KNN': KNN(),
                          'Support Vector Machines': SVM(),
                          'Decision Tree': DT(),
                          'Random Forest': RF(),
                          'Gradient Boosting': GB(),
                          'Extra Random Trees': ERT(),
                          'XGBoost': XGB(),
                          'Stochastic Gradient Descent': SGD(),
                          'Neural Network': NN()
                        }
        
#             func_dict  = {'Gaussian NB': GNB(),
#                           'Logistic Regression':LogisticReg()
#              }
    
            param_dict = {}

            for i in selected_models:
                param_dict[i] = func_dict[i]


            cardinality = 10

            # "Cardinality" means the number of unique values in a column
            # Select categorical columns with relatively low cardinality (convenient but arbitrary)
            categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < cardinality and 
                                    X_train_full[cname].dtype == "object"]

            # # Select numerical columns
            numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

            # # Keep selected columns only
            my_cols = categorical_cols + numerical_cols
            # X_train = pd.DataFrame(X_train_full[my_cols].copy(), columns = X.columns)
            # X_valid = pd.DataFrame(X_valid_full[my_cols].copy(), columns = X.columns)
            X_train = pd.DataFrame(X_train_full[my_cols].copy(), columns = my_cols)
            X_valid = pd.DataFrame(X_valid_full[my_cols].copy(), columns = my_cols)
            data_valid = pd.concat([X_valid, y_valid], axis = 1)

            # # Preprocessing for numerical data
            numerical_transformer = SimpleImputer(strategy='constant')

            # Preprocessing for categorical data
            categorical_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
            ])

            # Bundle preprocessing for numerical and categorical data
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numerical_transformer, numerical_cols),
                    ('cat', categorical_transformer, categorical_cols)
                ])
            #poly = Pipeline(steps = [('poly',PolynomialFeatures(2))])
            # poly = PolynomialFeatures(2)

            #Principal Component Analysis to get remove noisy features
            #pca = PCA(n_components = 100)

            from sklearn.base import BaseEstimator

            class MyClassifier(BaseEstimator):

                def __init__(
                    self, 
                    estimator = XGBClassifier(),
                ):
                    """
                    A Custom BaseEstimator that can switch between classifiers.
                    :param estimator: sklearn object - The classifier
                    """ 

                    self.estimator = estimator


                def fit(self, X, y=None, **kwargs):
                    self.estimator.fit(X, y)
                    return self


                def predict(self, X, y=None):
                    return self.estimator.predict(X)


                def predict_proba(self, X):
                    return self.estimator.predict_proba(X)


                def score(self, X, y):
                    return self.estimator.score(X, y)
                
                @property
                def classes_(self):
                    return self.estimator.classes_

            # Bundle preprocessing and modeling code in a pipeline
            my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),

                                          ('clf', MyClassifier())
                                         ])

            parameters = []

            for i in selected_models:
                parameters.append(param_dict[i])
            st.write(parameters)
            train = st.button('Train Model')
            if train:
                with st.spinner('Training Model...'):
                    from sklearn.model_selection import GridSearchCV
                    gscv = GridSearchCV(my_pipeline, parameters, cv=3, n_jobs=-1, return_train_score=False, verbose=3)
                    gscv.fit(X_train, y_train)
                    st.text('Best Parameters')
                    st.write(gscv.best_params_)
                    st.text('Best Score')
                    st.write(gscv.best_score_)
                    data_valid['Predicted'] = gscv.predict(X_valid)
                    st.write(data_valid)
                    cm = confusion_matrix(y_valid, gscv.predict(X_valid))  
                    st.text('Confusion Matrix')
                    fig1, ax1 = plt.subplots()
                    class_names = y_valid.unique()
                    confusion_matrix_plot(cm, class_names)
                    st.pyplot(fig1)
                    st.text('Performance Metrics')
                    st.write(combined_metrics(X_valid, y_valid, gscv))
                    df_test['Predicted'] = gscv.predict(df_test)
                    st.write(df_test)
#                     st.write(X_train)
                    explainer = shap.KernelExplainer(model_predict, X_train_full)
                    shap_values = explainer.shap_values(X_valid_full.iloc[2,:])
                    st.pyplot(shap.force_plot(explainer.expected_value[0], shap_values[0], X_valid_full.iloc[2,:], matplotlib=True, text_rotation=8))
                    
                    f = lambda x: model_predict(x)[:,1]
                    med = X_train_full.median().values.reshape((1,X_train_full.shape[1]))

                    explainer = shap.Explainer(f, med)
                    shap_values = explainer(X_valid_full.iloc[0:1000,:])
                    st.pyplot(shap.plots.waterfall(shap_values[10]))

                    pdp_ = pdp.pdp_isolate(model=gscv, dataset=X_train_full,
                                                model_features=X_train_full.columns,
                                                feature=my_cols[0])
#                     st.write(pdp_)
                    fig, axes = pdp.pdp_plot(pdp_isolate_out=pdp_, feature_name=my_cols[0], center = True, ncols=1, figsize = (15, 10))
                    st.pyplot(fig)


ModuleNotFoundError: No module named 'shap'

In [5]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_Classification.ipynb.
Converted 01_Regression.ipynb.
Converted index.ipynb.
